In [ ]:
import numpy as np
import pandas as pd
from keras import Sequential
from keras.src.callbacks import ModelCheckpoint
from keras.src.datasets import fashion_mnist
from keras.src.layers import Dense, Flatten, Dropout
from keras.src.utils import to_categorical
from matplotlib import pyplot as plt
from plotly import graph_objects as go
from plotly import express as px
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
(data, labels), (test_data, test_labels) = fashion_mnist.load_data()
data = data / 255.
test_data = test_data / 255.

data[0]

In [ ]:
def show_data(start_id):
    plt.figure(figsize=(16, 16))

    i = 1
    for img in test_data[start_id:start_id + 16]:
        plt.subplot(4, 4, i)
        plt.imshow(img, cmap="gray_r")

        if i > 16:
            break

        i += 1

    plt.show()


show_data(1)

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(units=256, activation="elu"))
model.add(Dense(units=10, activation="softmax"))

model.summary()

In [ ]:
mc = ModelCheckpoint("best_model_fashion.keras", monitor="val_accuracy", mode="max", save_best_only=True, verbose=1)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

r = model.fit(data, to_categorical(labels), validation_split=0.15, epochs=25, callbacks=[mc], batch_size=32 * 4)

model.save_weights("model_after_training.keras")

In [ ]:
def draw_history(r):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history["accuracy"], name="accuracy"))
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history["val_accuracy"], name="val_accuracy"))
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history["loss"], name="loss"))
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history["val_loss"], name="val_loss"))
    fig.show()


draw_history(r)

In [ ]:
model.load_weights("model_after_training.keras")

preds = model.predict(test_data)
preds = np.argmax(preds, axis=1)

df = pd.DataFrame({
    "true": test_labels,
    "predict": preds
})

df

In [ ]:
print(f"Accuracy: {accuracy_score(df.true, df.predict) * 100}%")

classes_indices = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag",
                   "Ankle boot"]

cm = confusion_matrix(df.true, df.predict)

px.imshow(cm, text_auto=True, x=["p_" + c for c in classes_indices], y=["t_" + c for c in classes_indices])

In [ ]:
indexes = np.where(df.true == 5)

plt.imshow(test_data[indexes[0][67]], cmap="gray_r")